In [1]:
import os

print(os.getcwd())
os.chdir('../')
print(os.getcwd())

c:\Users\Muhammed Nishan\OneDrive - Logic Plum, Inc\Documents\Learnings\MLOps Projects\datascienceproject\research
c:\Users\Muhammed Nishan\OneDrive - Logic Plum, Inc\Documents\Learnings\MLOps Projects\datascienceproject


In [2]:
# Creating Model trainer config

from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrinerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [3]:
# Updating Configuration manager

from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

class ConfigurationManager:
	def __init__(self, 
		config_filepath = CONFIG_FILE_PATH,
		params_filepath = PARAMS_FILE_PATH,
		schema_filepath = SCHEMA_FILE_PATH):
		
		# Loading configs, params and schema
		self.config = read_yaml(config_filepath)
		self.params = read_yaml(params_filepath)
		self.schema = read_yaml(schema_filepath)

		# Creating Artifacts
		create_directories([self.config.artifacts_root])
	
	def get_model_trainer_config(self) -> ModelTrinerConfig:
		config = self.config.model_trainer
		params = self.params.ElasticNet
		schema = self.schema.TARGET_COLUMN


		create_directories([config.root_dir])

		model_trainer_config = ModelTrinerConfig(
			root_dir = config.root_dir,
			train_data_path = config.train_data_path,
			test_data_path = config.test_data_path,
			model_name = config.model_name,
			alpha = params.alpha,
			l1_ratio = params.l1_ratio,
			target_column = schema.name
		)

		return model_trainer_config

In [4]:
# Component - ModelTrainer

import pandas as pd
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib

class ModelTrainer:
	def __init__(self, config: ConfigurationManager):
		self.config = config

	def train(self):

		train_data = pd.read_csv(self.config.train_data_path)
		test_data = pd.read_csv(self.config.test_data_path)

		train_x = train_data.drop([self.config.target_column], axis=1)
		test_x = test_data.drop([self.config.target_column], axis=1)
		train_y = train_data[[self.config.target_column]]
		test_y = test_data[[self.config.target_column]]

		lr = ElasticNet(
			alpha = self.config.alpha, 
			l1_ratio = self.config.l1_ratio,
			random_state = 42)
		lr.fit(train_x, train_y)
		
		joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [5]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-07-24 12:13:31,577: INFO: common: YAML File: config\config.yaml Loaded Successfully.]
[2025-07-24 12:13:31,592: INFO: common: YAML File: params.yaml Loaded Successfully.]
[2025-07-24 12:13:31,608: INFO: common: YAML File: schema.yaml Loaded Successfully.]
[2025-07-24 12:13:31,611: INFO: common: Created Directory at artifacts]
[2025-07-24 12:13:31,612: INFO: common: Created Directory at artifacts/model_trainer]
